In [1]:
#packages we need
using DifferentialEquations
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using DelimitedFiles
using FileIO

In [2]:
first_cell = 1
last_cell = 1000

1000

In [3]:
### Get plot times split into normal and quiescent cells NEW

function getCCtimes_nq(mut_folder, q)
    CC_times_df = DataFrame(G1S=Int64[], SG2=Int64[], G2M=Int64[], M=Int64[], Cell=Int64[], State=String[], Plotn=Int64[], Plotq=Int64[])

    for i in first_cell:last_cell
        if isfile("outputFiles_cell_cycle/"*mut_folder*"/plot_times/plot_times_"*string(i)*".csv") == true
            plot_times = DataFrame(CSV.File("outputFiles_cell_cycle/"*mut_folder*"/plot_times/plot_times_"*string(i)*".csv", header=true))
            plot_times[!, :Cell] = [i]
            plot_times[!, :State] = ["n"]
            plot_times[!, :Plotn] = plot_times[!, :M]
            plot_times[!, :Plotq] = [0]                    
            append!(CC_times_df, plot_times)
        elseif isfile("outputFiles_cell_cycle/"*mut_folder*"/quiescent/sol_df"*string(i)*".csv") == true
            plot_times = DataFrame(G1S=1, SG2=2, G2M=3, M=q)
            plot_times[!, :Cell] = [i]
            plot_times[!, :State] = ["q"]
            plot_times[!, :Plotn] = [0]
            plot_times[!, :Plotq] = plot_times[!, :M]
            append!(CC_times_df, plot_times)
        else
            continue
        end
    end
    return(CC_times_df)
end

getCCtimes_nq (generic function with 1 method)

In [4]:
CC_times_WT = getCCtimes_nq("WT", 4000)

,G1S,SG2,G2M,M,Cell,State,Plotn,Plotq
,Int64,Int64,Int64,Int64,Int64,String,Int64,Int64
1,621,896,1004,1048,1,n,1048,0
2,365,653,765,824,2,n,824,0
3,711,997,1095,1136,3,n,1136,0
4,207,492,601,645,4,n,645,0
5,425,711,816,858,5,n,858,0
6,436,719,815,855,6,n,855,0
7,357,623,728,767,7,n,767,0
8,234,527,633,677,8,n,677,0
9,487,791,891,935,9,n,935,0


In [5]:
CC_times_myc_mut = getCCtimes_nq("myc_mut", 4000)

In [5]:
function mergeCCtimes(CC_times_df, mut)
    CC_times_df_M = CC_times_df[!, [4,5,7,8]]
    rename!(CC_times_df_M, :M => mut)
    CC_times_WT_mut = innerjoin(CC_times_WT_M, CC_times_df_M, on = :Cell)
    sort!(CC_times_WT_mut, [:WT], rev=true)
    return(CC_times_WT_mut)
end

mergeCCtimes (generic function with 1 method)

In [ ]:
CC_times_WT_myc_mut = mergeCCtimes(CC_times_myc_mut, "myc_mut")

### Grid plot

In [11]:
pyplot()

Plots.PyPlotBackend()

In [10]:
using StatsPlots

In [12]:
function plotGrid(CC_times_comb, CC_times_mut, l1, l2, WT_col, mut_col, mut)

        ## select WT results    
        sort!(CC_times_comb, [:Plotn_WT], rev=true)
        CC_times_comb[!, :id] = collect(1:nrow(CC_times_comb))
    
        CC_times_WTn = CC_times_comb[(CC_times_comb.Plotn_WT .!= 0), :];
    
        CC_times_WT_q=CC_times_comb[(CC_times_comb.Plotq_WT .!= 0), :];
            
        ## select mut results
        CC_times_Mn = CC_times_comb[(CC_times_comb.Plotn .!= 0), :];
    
        CC_times_Mq=CC_times_comb[(CC_times_comb.Plotq .!= 0), :];

        ## bar plot 1
        plotb1=plot(ylim = (-10,1000))

        for row in eachrow(CC_times_Mn)
            plot!(plotb1,[0,row[mut]],[row["id"],row["id"]],color=mut_col,legend = false,lw=0.5)
        end    
        vline!([730], color=:grey, linestyle=:dash, lw=1, label="M WT 730 mins")
        plot!(CC_times_WTn[!, :Plotn_WT],CC_times_WTn[!, :id], color=WT_col, label=l1,lw=2)
        s=string.(collect(0:5:55))
        plot!(xticks = (0:(60*5):(60*55),s),xlabel="time (hours)",ylabel="cell number",fmt = :png)

    
        ## bar plot 2
        plotb2=plot(yaxis=false,ylim = (-10,1000),xticks=false)
        for row in eachrow(CC_times_WT_q)
            plot!(plotb2,[0,row["Plotq_WT"]],[row["id"],row["id"]],color=WT_col,legend = false)
        end

    
        ## bar plot 3
        plotb3=plot(yaxis=false,ylim = (-10,1000),xticks=false)
        for row in eachrow(CC_times_Mq)
            plot!(plotb3,[0,row["Plotq"]],[row["id"],row["id"]],color=mut_col,legend = false)
        end
        
        ## density
        plotd = density(CC_times_WTn[!, :Plotn_WT], label = l1, alpha=0.5, color=WT_col)
                density!(CC_times_Mn[!, :Plotn], label = l2, alpha=0.5, color=mut_col)
                s=string.(collect(0:5:55))
                plot!(xticks = (0:(60*5):(60*55),s),xlabel="time (hours)",ylabel="density",fmt = :png)
    
        ## bar plot 4
        q_res1 = [nrow(CC_times_comb[(CC_times_comb.Plotq_WT .!= 0), :]),0]
        q_res2 = [0, nrow(CC_times_comb[(CC_times_comb.Plotq .!= 0), :])]
        #qres = 
        plotb4 = bar(q_res1,
                    color=WT_col,
                    bar_width=0.5,
                    label=false,
                    ylim = (0,200),
                    xticks=(1:length([l1,l2]),[l1,l2]),
                    linecolor = :match
                    )
                 bar!(q_res2, color=mut_col, label=false, bar_width=0.5,linecolor = :match)
    
        l = @layout [
        a{0.5h,0.6w} b{0.5h,0.01w} c{0.5h,0.01w}
        d{0.4h,0.62w} e{0.4h,0.1w}]
    
        plot_comb = plot(plotb1, plotb2, plotb3, plotd, plotb4, 
                    layout=l, size=(1000,1500))
        return(plot_comb, plotb1, plotb2, plotd, plotb4)
end

plotGrid (generic function with 1 method)

In [17]:
plots_WT_myc = plotGrid(CC_times_WT_myc_mut, CC_times_myc_mut, "WT", "Myc x1.5", :black, :green, "myc_mut")

(Plot{Plots.PyPlotBackend() n=1079}, Plot{Plots.PyPlotBackend() n=979}, Plot{Plots.PyPlotBackend() n=81}, Plot{Plots.PyPlotBackend() n=2}, Plot{Plots.PyPlotBackend() n=2})